# IMDbの映画レビューセットを用いた感情分析

ここで扱う映画レビューデータセットは、「肯定的」か「否定的」かで分類される50000件の映画レビューで構成されている。  
これらの映画レビューのサブセットから意味のある情報を抽出し、  
レビューする人が映画を「好き」か「嫌い」のどちらを評価したのかを予測する機械学習モデルを構築する。

In [1]:
from typing import List, Dict
import pandas
from pandas.core.frame import DataFrame
import os
import numpy
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from tqdm import tqdm
import pickle

## 前処理

### データを読み込み、一つのCSVファイルに出力する

IMDb(Intenet Movie Database)から映画レビューデータセットを取り出す。  
データは[こちら](http://ai.stanford.edu/~amaas/data/sentiment/)から取得した。  
以下を実行する前に、./data/aclImdb_v1.tar.gzを解凍する必要がある。

In [69]:
path_base: str = "./data/aclImdb/"
path_test: str = path_base + "test/"
path_train: str = path_base + "train/"
neg: str = "neg"
pos: str = "pos"
path_test_negative: str = path_test + neg
path_test_positive: str = path_test + pos
path_train_negative: str = path_train + neg
path_train_positive: str = path_train + pos

df: DataFrame = pandas.DataFrame()


def load_data(path: str, label: int, data_frame: DataFrame) -> DataFrame:
    for file in tqdm(os.listdir(path)):
        with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
            text: str = infile.read()
        data_frame = data_frame.append([[text, label]], ignore_index=True)
    return data_frame


df = load_data(path=path_test_negative, label=0, data_frame=df)
df = load_data(path=path_test_positive, label=1, data_frame=df)
df = load_data(path=path_train_negative, label=0, data_frame=df)
df = load_data(path=path_train_positive, label=1, data_frame=df)

df.columns = ["review", "sentiment"]

100%|██████████| 12500/12500 [00:45<00:00, 275.44it/s]


データをシャッフルして、CSVファイルに出力する

In [74]:
numpy.random.seed(0)
df = df.reindex(numpy.random.permutation(df.index))
df.to_csv("./data/movie_data.csv", index=False)

### テキストデータ内の単語を特徴ベクトルに変換する

文章や単語などのカテゴリーデータは、機械学習アルゴリズムに渡す前に数値に変換する必要がある。  

ここでは、テキストを数値の特徴ベクトルとして表現できるBoW(Bag of Words)モデルを用いる。  
BoWモデルの基本的な考え方は以下のとおりである。  
1. ドキュメントの集合全体から例えば単語という一意なトークン(token)からなる語彙を作成する。
2. 各ドキュメントでの各単語の出現回数を含んだ特徴ベクトルを構築する。  

#### テキストデータをクレンジングする

モデルを構築する前には、不要な文字を取り除く必要がある。  
ここではHTMLマークアップ、顔文字以外の句読点を削除する。

クレンジングの処理は以下のcleanse関数にて定義する。

In [102]:
def cleanse(text: str) -> str:
    text = re.sub("<[^>]*>", '', text)
    emoticons: List[str] = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+]", ' ', text.lower()) + ''.join(emoticons).replace("-", '')

    return text

#### ストップワードの除去を行う

ストップワード(ドキュメントの判別に有効ではない単語、is, and, hasなど)の除去を行う。  
レビューデータからストップワードを除去するには、NLTKライブラリで提供されている127個の英語のストップワードを使用する。


以下では、nltkでストップワードのデータのダウンロードを行い、  
pickeで英語に関するストップワードのデータをバイトコードにシリアライズする。  
ここではすでにシリアライズ済みであるので、下記の実行の必要はない。

In [81]:
nltk.download("stopwords")

path_pkl: str = os.path.join("./data/", "pkl_objects")

if not os.path.exists(path_pkl):
    os.makedirs(path_pkl)
    
pickle.dump(stopwords.words("english"),
            open(os.path.join(path_pkl, "stopwords.pkl"), "wb"),
            protocol=4)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fukunaritakeshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ストップワードのデシリアライズを行うには、下記を実行する

In [103]:
stopwords_reloaded: List[str] = pickle.load(open(os.path.join(path_pkl, "stopwords.pkl"), "rb"))

#### ステミングを行う

トークン化にはワードステミング(Word Stemming)を用いる。  
これは単語を原型に変換することで、関連する単語を同じ語幹にマッピングするようなプロセスである。  
その中でもPorterステミングを用いる。

以上のクレンジング、ストップワードの除去、ステミングの一連の処理を以下のtokeninzer関数で定義する。  
この関数は次節のHashingVectorizerにて使用する。

In [104]:
def tokenizer(text: str):
    cleanse(text)
    porter_stemmer: PorterStemmer = PorterStemmer()

    return [porter_stemmer.stem(word) for word in text.split()
            if word not in stopwords_reloaded]

## モデルをトレーニングする

ここではアウトオブコア学習+ミニバッチ学習を適用する。
アウトオブコア学習とは、

### ベクタライザを用いる

ここではHashingVectorizerを用いる。  
(アウトオブコア学習でなければ、CountingVectorizerを用いることが可能である。)

また、ここで前節のtokenizer関数を用いている。

In [105]:
hashing_vector: HashingVectorizer = HashingVectorizer(decode_error='ignore',
                                                      n_features=2**21,
                                                      preprocessor=None,
                                                      tokenizer=tokenizer)

### ロジスティック回帰分類器を初期化する
**SGDClassifierをなぜ使うのか**

In [106]:
sgd_classifier: SGDClassifier = SGDClassifier(loss='log', random_state=1, max_iter=1)

ジェネレータ関数を定義する。
  
ドキュメントを１つずつ読み込んで返す。

In [107]:
def stream(path: str) -> (str, int):
    with open(path, 'r', encoding='utf-8') as file:
        # ヘッダーを読み飛ばす
        next(file)
        for line in file:
            text: str = line[:-3]
            label: int = int(line[-2]) # 一行の最後尾は'\n'のため
            yield text, label

ミニバッチ関数を定義する。

stream_documents関数からドキュメントストリームを受け取り、  
size引数によって指定された個数のドキュメントを返す。

In [108]:
def get_minibatch(documents_stream, size: int):
    documents: List[str] = []
    y: List[int] = []
    try:
        for _ in range(size):
            text, label = next(documents_stream)
            documents.append(text)
            y.append(label)
    except StopIteration:
        print("StopIteration")
        return None, None
    return documents, y


アウトオブコア学習を行う。

1000個ずつデータを読み込み、それを学習に用いる。  
これを45回繰り返す(ミニバッチ法)。  
つまり45000個を訓練データに用いて、残り5000個をテストデータとして用いる。

In [109]:
classes: numpy.ndarray = numpy.array([0, 1])
document_stream = stream(path="./data/movie_data.csv")

In [110]:
for index in tqdm(range(45)):
    X_train, y_train = get_minibatch(documents_stream=document_stream, size=1000)

    if not X_train:
        print("breaked...")
        break

    X_train_vectorized = hashing_vector.transform(X_train)
    sgd_classifier.partial_fit(X=X_train_vectorized, y=y_train, classes=classes)

100%|██████████| 45/45 [02:46<00:00,  3.70s/it]


## 学習済みモデルに対して予測を行う

In [111]:
X_test, y_test = get_minibatch(documents_stream=document_stream, size=5000)
X_test = hashing_vector.transform(X_test)
accuracy: float = sgd_classifier.score(X_test, y_test)

print("正解率: ", accuracy)

正解率:  0.845


## 学習済みの推定器に対して保存やリロードをする

学習済みの推定器を、バイトコードにシリアライズして保存する。

In [112]:
if not os.path.exists(path_pkl):
    os.makedirs(path_pkl)
    
pickle.dump(sgd_classifier,
            open(os.path.join(path_pkl, "classifier.pkl"), "wb"),
            protocol=4)

学習済みの推定器をデシリアライズする

In [113]:
classifier_reloaded: SGDClassifier = pickle.load(open(os.path.join(path_pkl, "classifier.pkl"), "rb"))

In [115]:
label: Dict[int, str] = {0: "negative", 1: "positive"}

example: List[str] = ["I love this movie"]
X = hashing_vector.transform(example)

print("Predict: ", label[classifier_reloaded.predict(X)[0]])
print("Probability: ", numpy.max(classifier_reloaded.predict_proba(X)*100))

Predict:  positive
Probability:  91.331173329
